In [1]:
!pip install transformers
!pip install spacy-transformers
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 16.3 MB/s eta 0:00:00
✔ Download

In [8]:
from google.colab import drive
import json
import spacy
import spacy_transformers
import pandas as pd
import os

In [3]:
# Load spaCy's English model
nlp = spacy.load("en_core_web_trf")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

In [4]:
# Mount
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
working_dir = '/content/drive/My Drive/deep_learning_project/submition/data/Winoground'

In [6]:
def get_full_phrase(token):
    """Helper function to return the full phrase for a given token, including its modifiers."""
    # Collect the token and its modifiers
    phrase_tokens = [token]

    # Add left children (modifiers, determiners, etc.) and sort by token position
    phrase_tokens.extend(list(token.lefts))

    # Sort tokens by their position in the sentence to maintain proper order
    phrase_tokens = sorted(phrase_tokens, key=lambda x: x.i)

    # Return the full phrase
    return ' '.join([t.text for t in phrase_tokens])

def identify_roles(sentence):
    doc = nlp(sentence)
    agent = ""
    patient = ""
    action = ""
    remainder = []

    agent_tokens = set()
    patient_tokens = set()
    action_token = None

    for token in doc:
        # Identify subject (agent) and object (patient)
        if token.dep_ == "nsubj":  # Nominal subject (agent)
            agent = get_full_phrase(token)
            agent_tokens.update([token] + list(token.lefts))  # Collect tokens for agent
        elif token.dep_ == "dobj":  # Direct object (patient)
            patient = get_full_phrase(token)
            patient_tokens.update([token] + list(token.lefts))  # Collect tokens for patient
        elif token.pos_ == "VERB":  # Action (verb)
            action = token.text
            action_token = token  # Store the action token
        else:
            remainder.append(token)  # Collect other tokens as remainder

    # Exclude agent, patient, and action tokens from remainder
    remainder = [token.text for token in remainder if token not in agent_tokens and token not in patient_tokens and token != action_token]

    # Remove any period ('.') from the remainder list
    remainder = [token for token in remainder if token != '.']

    # Return agent, action, patient, and the remainder
    return agent, action, patient, remainder

In [9]:
winoground_data = []
with open(os.path.join(working_dir, 'winoground.jsonl'), 'r') as file:
    for line in file:
        winoground_data.append(json.loads(line))
winoground_dict = {item['id']: item for item in winoground_data}


In [10]:
filtered_winoground_dict = {}
filtered_winoground_df = pd.DataFrame(columns=['id', 'caption_0', 'image_0', 'caption_1', 'image_1'])

for key in winoground_dict.keys():
  agent_0, action_0, patient_0, remainder_0 = identify_roles(winoground_dict[key]['caption_0'])
  agent_1, action_1, patient_1, remainder_1 = identify_roles(winoground_dict[key]['caption_1'])
  if agent_0 != "" and action_0 != "" and patient_0 != "" and agent_1 != "" and action_1 != "" and patient_1 != "":
    new_row = {'id': winoground_dict[key]['id'], 'caption_0': winoground_dict[key]['caption_0'], 'image_0': winoground_dict[key]['image_0'],
               'caption_1': winoground_dict[key]['caption_1'], 'image_1': winoground_dict[key]['image_1']}
    filtered_winoground_df = pd.concat([filtered_winoground_df, pd.DataFrame([new_row])], ignore_index=True)

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [12]:
winoground_filtering_output_filter = f"{working_dir}/../../output/winoground_filtering_output"
os.makedirs(winoground_filtering_output_filter, exist_ok=True)
filtered_winoground_df.to_csv(os.path.join(winoground_filtering_output_filter, "winoground_filtering_output.csv"), index=False)
filtered_winoground_df.to_json(os.path.join(winoground_filtering_output_filter, "winoground_filtering_output.json"), orient='records', lines=True)
